In [33]:
# import stuff here
import requests
import pymongo
from bs4 import BeautifulSoup
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd


In [34]:
# Set up splinter
executable_path = {'executable_path':ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Trying to download new driver from https://chromedriver.storage.googleapis.com/89.0.4389.23/chromedriver_win32.zip
[WDM] - Driver has been saved in cache [C:\Users\Sharon\.wdm\drivers\chromedriver\win32\89.0.4389.23]


In [35]:
nasa_url = "https://redplanetscience.com/"

In [36]:
browser.visit(nasa_url)

In [40]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
#results = soup.find_all('div', class_='content_title')  THIS WORKS
results = soup.find_all('div', class_='list_text')

#results

In [ ]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

db = client.nasa_db
collection = db.items


# gather up the items for mongo db
# if (title and blurb):
#    post = {
#        'title': title,
#        'blurb': blurb,
#     }

#    collection.insert_one(post)

In [42]:
for result in results:
    try:
        title = result.find('div', class_='content_title').text
        
        blurb = result.find('div', class_='article_teaser_body').text
        
        print('**************')
        print(f'Title: {title}')
        print(f'Blurb: {blurb}')
        
    except Exception as e:
        print(e)
    

**************
Title: Mars Scientists Investigate Ancient Life in Australia
Blurb: Teams with NASA's Mars 2020 and ESA's ExoMars practiced hunting for fossilized microbial life in the Australian Outback in preparation for their Red Planet missions. 
**************
Title: NASA Moves Forward With Campaign to Return Mars Samples to Earth
Blurb: During this next phase, the program will mature critical technologies and make critical design decisions as well as assess industry partnerships.
**************
Title: 3 Things We've Learned From NASA's Mars InSight 
Blurb: Scientists are finding new mysteries since the geophysics mission landed two years ago.
**************
Title: Mars Helicopter Attached to NASA's Perseverance Rover
Blurb: The team also fueled the rover's sky crane to get ready for this summer's history-making launch.
**************
Title: NASA Updates Mars 2020 Mission Environmental Review
Blurb: NASA and the Department of Energy have completed a more detailed risk analysis for 

In [26]:
# response = requests.get("https://redplanetscience.com/")
# response

<Response [200]>

In [32]:
# NASA Mars News
# Scrape the Mars News Site, collect latest News Title and Paragraph Text. 
#soup = BeautifulSoup(page.content, 'html.parser')
# soup = BeautifulSoup(response.text, 'lxml')
# print(soup.prettify())

In [31]:
# results = soup.find_all('div', class_='content_title')   #.find('a').text
# print(len(results))

0


In [107]:
# results = soup.find_all('section', class_='image_and_description_container')
# print(len(results))
# soup.body.section
# sections = soup.find_all('section', class_='image_and_description_container')

# for section in sections:
#     print(section.find('div', class_='article_teaser_body'))

1
None


In [66]:
# for result in results:
#     #print(result)
#     try:
#         #title = result.find('div', class_='content_title').text
#         title = 
#         blurb = soup.find_all('div', class_='article_teaser_body')
        
#         if (title and blurb):
#             print('------------')
#             print(title)
#             print(blurb)
#     except AttributeError as e:
#         print(e)

'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'


In [ ]:
headlines = []

for div in divs:
    if (div.)

In [43]:

divList = soup.body.find_all('section', class_='image_and_description_container')
len(divList)

1

In [41]:
# paragraphs = soup.find_all('section', class_='image_and_description_container')
# for paragraph in paragraphs:
#     print(paragraph.text)

In [42]:
results = soup.find_all('div', class_='list_text')
for result in results:
    print(result)

In [ ]:
#list(soup.children)
print(soup.prettify())

In [ ]:
[type(item) for item in list(soup.children)]

In [ ]:
html = list(soup.children)[2]
html

In [ ]:
list(html.children)

In [ ]:
body = list(html.children)[3]
body

In [ ]:
p = list(body.children)[1]

# for kid in p:
#     p.get_text()

In [ ]:
p.get_text()

In [ ]:
soup.find_all('p')[0].get_text()

In [ ]:
x = soup.find_all('p')
x

In [ ]:
# JPL Mars Space Images - Featured Image